In [11]:
import uuid
import os
import tempfile

import ffmpeg
# NOTE: need to make sure ffmpeg cli is installed

In [12]:
def make_video_clip(input_video_filename, output_video_filename, start_time, end_time):
    ffmpeg.input(input_video_filename, ss=start_time, to=end_time).output(output_video_filename).run(overwrite_output=True)

In [13]:
def combine_timestamps(input_video_filename, output_video_filename, timestamp_windows):
    # create clips of video for timestamp windows and save to temp dir
    # combine clips into video
    with tempfile.TemporaryDirectory() as tempdir:
        # generate clips from source video by time stamp window
        outfiles = []
        
        for window in timestamp_windows:
            out = f"{tempdir}/{str(uuid.uuid4())[:8]}.mp4"

            make_video_clip(input_video_filename, out, window[0], window[1])
            outfiles.append(f"file {out}")

        # Combine outfile paths into a txt file
        combined_file = f"{tempdir}/{str(uuid.uuid4())[:8]}.txt"

        with open(combined_file, "w") as fp:
            fp.write("\n".join(outfiles))

        # Combine clips using source files
        ffmpeg.input(combined_file, format='concat', safe=0).output(output_video_filename, c='copy').run(overwrite_output=True)

In [14]:
import pymongo

In [15]:
pymongo_client = pymongo.MongoClient(os.environ["MONGO_URI"])

In [16]:
collection = pymongo_client["video-texts"]["messi-video"]

In [24]:
import json
with open("search.json") as fp:
    search_query = json.load(fp)

print(search_query)

{'$vectorSearch': {'queryVector': [0.015075684, 0.07409668, -0.0040740967, 0.04562378, 0.021469116, -0.0015964508, -0.03756714, -0.003993988, -0.02217102, 0.0063667297, -0.055419922, 0.056610107, 0.019851685, -0.03591919, 0.058013916, -0.05029297, -0.009536743, -0.015808105, -0.010223389, 0.0713501, -0.021377563, -0.0317688, 0.00945282, -0.051940918, 0.03353882, -0.01675415, -0.01701355, -0.02709961, -0.07409668, 0.019500732, 0.022460938, -0.039489746, -0.0011730194, -0.052856445, 0.0067329407, 0.03479004, 0.03112793, -0.051483154, 0.0046463013, -0.00642395, 0.035369873, 0.004299164, -0.009086609, -0.0047721863, 0.011665344, -0.032196045, 0.021957397, -0.024246216, 0.04650879, -0.04525757, -0.04751587, 0.024902344, -0.007347107, -0.020111084, 0.020889282, 0.010917664, 0.047332764, 0.010025024, 0.029571533, -0.010734558, 0.03201294, 0.023635864, -0.056274414, 0.014144897, 0.02218628, -0.033569336, -0.019363403, -0.013427734, -0.015792847, -0.052337646, 0.024658203, -0.0007004738, 0.0256

In [30]:
pipeline = [search_query]
results = collection.aggregate(pipeline)
print(results)

In [32]:
results_as_dict = list(results)
print(results_as_dict)

[{'_id': ObjectId('6611cd9883e5d22716a3342c'), 'ts': 570, 'vector-embedding': [0.009521484, 0.0524292, 0.003868103, -0.038726807, -0.036193848, -0.011367798, -0.04284668, -0.025878906, 0.041381836, 0.0051956177, -0.02859497, 0.021713257, 0.011253357, -0.0149383545, -0.006855011, 0.008201599, -0.020187378, 0.030670166, 0.039733887, -0.013343811, -0.009811401, 0.0009851456, 0.018707275, -0.041259766, 0.019897461, 0.032318115, -0.0030956268, -0.055419922, -0.020980835, -0.010574341, -0.0010890961, 0.030578613, 0.040100098, -0.06500244, 0.05987549, 0.033172607, 0.0028495789, -0.044952393, 0.033203125, -0.0046081543, -0.0134887695, 0.008522034, -0.041656494, -0.008766174, -0.019836426, -0.0390625, -0.009651184, -0.012802124, -0.009796143, -0.043395996, -0.021362305, 0.0006055832, -0.021820068, 0.061767578, 0.034118652, 0.012252808, 0.024002075, -0.049041748, 0.04208374, 0.023925781, 0.009017944, -8.034706e-05, -0.070373535, 0.0079956055, -0.004798889, 0.057159424, -0.04321289, -0.03918457, 

In [34]:
timestamps = []
for result in results_as_dict:
    timestamps.append(result["ts"])

sT = sorted(timestamps)
sT = [i/2 for i in sT]
print(sT)




[5.5, 6.0, 7.0, 7.5, 10.0, 10.5, 11.0, 11.5, 12.0, 84.0, 88.0, 92.0, 109.5, 110.5, 111.0, 158.0, 162.5, 186.5, 187.0, 191.0, 192.5, 219.5, 222.5, 223.0, 223.5, 224.0, 226.5, 249.5, 253.0, 253.5, 254.5, 255.0, 255.5, 256.0, 257.5, 258.0, 258.5, 278.0, 283.0, 283.5, 284.0, 285.0, 286.5, 287.0, 288.0, 288.5, 289.0, 289.5, 290.0]


In [39]:
import datetime

final_timestamps = []
for timestamp in sT:
    final_timestamps.append((str(datetime.timedelta(seconds=timestamp)), str(datetime.timedelta(seconds=timestamp + 0.5))))

In [40]:
print(final_timestamps)

[('0:00:05.500000', '0:00:06'), ('0:00:06', '0:00:06.500000'), ('0:00:07', '0:00:07.500000'), ('0:00:07.500000', '0:00:08'), ('0:00:10', '0:00:10.500000'), ('0:00:10.500000', '0:00:11'), ('0:00:11', '0:00:11.500000'), ('0:00:11.500000', '0:00:12'), ('0:00:12', '0:00:12.500000'), ('0:01:24', '0:01:24.500000'), ('0:01:28', '0:01:28.500000'), ('0:01:32', '0:01:32.500000'), ('0:01:49.500000', '0:01:50'), ('0:01:50.500000', '0:01:51'), ('0:01:51', '0:01:51.500000'), ('0:02:38', '0:02:38.500000'), ('0:02:42.500000', '0:02:43'), ('0:03:06.500000', '0:03:07'), ('0:03:07', '0:03:07.500000'), ('0:03:11', '0:03:11.500000'), ('0:03:12.500000', '0:03:13'), ('0:03:39.500000', '0:03:40'), ('0:03:42.500000', '0:03:43'), ('0:03:43', '0:03:43.500000'), ('0:03:43.500000', '0:03:44'), ('0:03:44', '0:03:44.500000'), ('0:03:46.500000', '0:03:47'), ('0:04:09.500000', '0:04:10'), ('0:04:13', '0:04:13.500000'), ('0:04:13.500000', '0:04:14'), ('0:04:14.500000', '0:04:15'), ('0:04:15', '0:04:15.500000'), ('0:04:

In [53]:
start_time_1 = '00:00:00' # Start time for trimming (HH:MM:SS)
end_time_1 = '00:00:5' # End time for trimming (HH:MM:SS)
start_time_2 = '00:00:10' # Start time for trimming (HH:MM:SS)
end_time_2 = '00:00:15' # End time for trimming (HH:MM:SS)

timestamps = []

combine_timestamps("messi.mp4", "out.mp4", final_timestamps)

ffmpeg version 6.1.1 Copyright (c) 2000-2023 the FFmpeg developers
  built with Apple clang version 14.0.0 (clang-1400.0.29.202)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/6.1.1_7 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopenvino --enable-li

In [54]:
def combine_audio_video(input_video_file, input_audio_file, output_video_file):
    input_video = ffmpeg.input(input_video_file)
    input_audio = ffmpeg.input(input_audio_file)

    ffmpeg.concat(input_video, input_audio, v=1, a=1).output(output_video_file).run()